In [17]:
import pandas as pd
from email import message_from_string
from email.parser import Parser

In [18]:
data = pd.read_csv('E:\\Data Archive\\Datasets\\Enron Emails\\csv\\emails.csv', usecols=['message'])
data

,message
0,Message-ID: <18782981.1075855378110.JavaMail.e...
1,Message-ID: <15464986.1075855378456.JavaMail.e...
2,Message-ID: <24216240.1075855687451.JavaMail.e...
3,Message-ID: <13505866.1075863688222.JavaMail.e...
4,Message-ID: <30922949.1075863688243.JavaMail.e...
...,...
517396,Message-ID: <26807948.1075842029936.JavaMail.e...
517397,Message-ID: <25835861.1075842029959.JavaMail.e...
517398,Message-ID: <28979867.1075842029988.JavaMail.e...
517399,Message-ID: <22052556.1075842030013.JavaMail.e...


In [19]:
data['id'] = data.index

In [20]:
data

,message,id
0,Message-ID: <18782981.1075855378110.JavaMail.e...,0
1,Message-ID: <15464986.1075855378456.JavaMail.e...,1
2,Message-ID: <24216240.1075855687451.JavaMail.e...,2
3,Message-ID: <13505866.1075863688222.JavaMail.e...,3
4,Message-ID: <30922949.1075863688243.JavaMail.e...,4
...,...,...
517396,Message-ID: <26807948.1075842029936.JavaMail.e...,517396
517397,Message-ID: <25835861.1075842029959.JavaMail.e...,517397
517398,Message-ID: <28979867.1075842029988.JavaMail.e...,517398
517399,Message-ID: <22052556.1075842030013.JavaMail.e...,517399


In [21]:
def parse_email(email_content: str) -> tuple:
    email = message_from_string(email_content)
    sender = email['from']
    recipient = email['to']
    subject = email['subject']
    cc = email['cc']

    body = ''

    if email.is_multipart():
        for part in email.walk():
            ctype = part.get_content_type()
            cdispo = str(part.get('Content-Disposition'))

            if ctype == 'text/plain' and 'attachment' not in cdispo:
                body = part.get_payload(decode=True) 
                break
            # endif
        # endfor
    # endif
    else:
        body = email.get_payload(decode=True)
    # endelse

    return sender, recipient, subject, cc, body.decode('utf-8')
# endfunc

In [22]:
def filter_recipient_email(rec: str) -> list | str:
    if rec is None:
        return 'None'
    # endif

    recips = rec.split(', ')

    filtered_recips = [r for r in recips if '@enron.com' in r]

    return filtered_recips if len(filtered_recips) > 0 else 'None'
# endif

In [23]:
data[['sender', 'recipient', 'subject', 'cc', 'body']] = data['message'].apply(
    lambda email: pd.Series(parse_email(email))
)

data.drop(columns=['message'], inplace=True)
data

,id,sender,recipient,subject,cc,body
0,0,phillip.allen@enron.com,tim.belden@enron.com,,None,Here is our forecast\n\n
1,1,phillip.allen@enron.com,john.lavorato@enron.com,Re:,None,Traveling to have a business meeting takes the...
2,2,phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,None,test successful. way to go!!!
3,3,phillip.allen@enron.com,randall.gay@enron.com,,None,"Randy,\n\n Can you send me a schedule of the s..."
4,4,phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,None,Let's shoot for Tuesday at 11:45.
...,...,...,...,...,...,...
517396,517396,john.zufferli@enron.com,kori.loibl@enron.com,Trade with John Lavorato,None,This is a trade with OIL-SPEC-HEDGE-NG (John L...
517397,517397,john.zufferli@enron.com,john.lavorato@enron.com,Gas Hedges,None,Some of my position is with the Alberta Term b...
517398,517398,john.zufferli@enron.com,dawn.doucet@enron.com,RE: CONFIDENTIAL,None,2\n\n -----Original Message-----\nFrom: \tDouc...
517399,517399,john.zufferli@enron.com,jeanie.slone@enron.com,Calgary Analyst/Associate,None,Analyst\t\t\t\t\tRank\n\nStephane Brodeur\t\t\...


In [24]:
data['recipient'] = data['recipient'].apply(filter_recipient_email)
data['cc'] = data['cc'].apply(filter_recipient_email)
data = data[data['recipient'] != 'None']
data

,id,sender,recipient,subject,cc,body
0,0,phillip.allen@enron.com,[tim.belden@enron.com],,None,Here is our forecast\n\n
1,1,phillip.allen@enron.com,[john.lavorato@enron.com],Re:,None,Traveling to have a business meeting takes the...
2,2,phillip.allen@enron.com,[leah.arsdall@enron.com],Re: test,None,test successful. way to go!!!
3,3,phillip.allen@enron.com,[randall.gay@enron.com],,None,"Randy,\n\n Can you send me a schedule of the s..."
4,4,phillip.allen@enron.com,[greg.piper@enron.com],Re: Hello,None,Let's shoot for Tuesday at 11:45.
...,...,...,...,...,...,...
517392,517392,john.zufferli@enron.com,"[nicole.laporte@enron.com, paul.dunsmore@enron...",RE: Counterparty List,None,"Nicole and everyone, I think the directory is ..."
517396,517396,john.zufferli@enron.com,[kori.loibl@enron.com],Trade with John Lavorato,None,This is a trade with OIL-SPEC-HEDGE-NG (John L...
517397,517397,john.zufferli@enron.com,[john.lavorato@enron.com],Gas Hedges,None,Some of my position is with the Alberta Term b...
517398,517398,john.zufferli@enron.com,[dawn.doucet@enron.com],RE: CONFIDENTIAL,None,2\n\n -----Original Message-----\nFrom: \tDouc...


In [25]:
data = data[data['sender'].str.contains('@enron.com')]
data

,id,sender,recipient,subject,cc,body
0,0,phillip.allen@enron.com,[tim.belden@enron.com],,None,Here is our forecast\n\n
1,1,phillip.allen@enron.com,[john.lavorato@enron.com],Re:,None,Traveling to have a business meeting takes the...
2,2,phillip.allen@enron.com,[leah.arsdall@enron.com],Re: test,None,test successful. way to go!!!
3,3,phillip.allen@enron.com,[randall.gay@enron.com],,None,"Randy,\n\n Can you send me a schedule of the s..."
4,4,phillip.allen@enron.com,[greg.piper@enron.com],Re: Hello,None,Let's shoot for Tuesday at 11:45.
...,...,...,...,...,...,...
517392,517392,john.zufferli@enron.com,"[nicole.laporte@enron.com, paul.dunsmore@enron...",RE: Counterparty List,None,"Nicole and everyone, I think the directory is ..."
517396,517396,john.zufferli@enron.com,[kori.loibl@enron.com],Trade with John Lavorato,None,This is a trade with OIL-SPEC-HEDGE-NG (John L...
517397,517397,john.zufferli@enron.com,[john.lavorato@enron.com],Gas Hedges,None,Some of my position is with the Alberta Term b...
517398,517398,john.zufferli@enron.com,[dawn.doucet@enron.com],RE: CONFIDENTIAL,None,2\n\n -----Original Message-----\nFrom: \tDouc...
